In [1]:
from google import genai
import os
from google.genai import types

In [2]:
client = genai.Client(
      vertexai=True
  )


In [3]:
client

In [4]:
client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Is it too late to join the course?"
)
print(response.text)

Whether or not it's too late to join a course depends on a few factors. To give you the best answer, I need a little more information! Please tell me:

*   **What course are you referring to?** (Include the course name and where it's offered).
*   **When did the course start or when is the next start date?**
*   **What is the course provider's policy on late enrollment?** (e.g. the university, website, etc.)
*   **What are your specific concerns about joining late?** (e.g. missed material, deadlines, etc.)

The sooner you give me the details, the sooner I can assist you.


In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-15 16:21:47--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-15 16:21:47 (62.2 MB/s) - ‘minsearch.py.2’ saved [4073/4073]



In [6]:
import minsearch

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [7]:
minsearch

<module 'minsearch' from '/workspaces/llm-zoomcamp/01-intro/minsearch.py'>

In [8]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-06-15 16:21:49--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.3’

documents.json.3    100%[===================>] 642.90K  --.-KB/s    in 0.004s  

2025-06-15 16:21:49 (169 MB/s) - ‘documents.json.3’ saved [658332/658332]



In [9]:
import json

In [10]:
with open('documents.json', 'rt') as f_in:
    docs_raw  = json.load(f_in)

In [11]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [12]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields=["course"]
)

In [14]:
index.fit(documents)

In [15]:
q = "The course has already started. Can I still enroll?"

In [16]:
boost = {"question": 3.0, "section": 0.5}

results = index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [17]:
client

In [18]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=q
)
print(response.text)

Whether or not you can still enroll in a course that has already started depends on several factors:

*   **The course provider's policy:** Some institutions or platforms have strict deadlines for enrollment, while others are more flexible.
*   **The type of course:** Self-paced courses are often open for enrollment even after the start date. Live, instructor-led courses may have a shorter or no grace period.
*   **The course duration:** A shorter course may have a stricter enrollment deadline than a longer one.
*   **The amount of material already covered:** If the course is very far along, it may be difficult to catch up, and the provider might discourage or prevent enrollment.
*   **Availability:** If the course has reached capacity, enrollment may be closed regardless of the start date.

**To find out if you can still enroll, you should:**

1.  **Check the course website or platform:** Look for information about enrollment deadlines or policies.
2.  **Contact the course provider di

In [19]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [20]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [21]:
prompt = prompt_template.format(question=q, context=context).strip()

In [22]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: The course has already started. Can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your 

In [23]:
response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents=prompt
)
print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.



In [24]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [25]:
search('how do i run kafka?')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output NONE.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [27]:
def llm(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash", 
        contents=prompt
    )
    return response.text

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
print(rag('how do I run kafka?'))

To run kafka, refer to these instructions depending on the language you are using:
*   **Java:** In the project directory, run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`
*   **Python:** Create a virtual environment and run requirements.txt and the python files in that environment.
    *   To create a virtual env and install packages (run only once):
        `python -m venv env`
        `source env/bin/activate`
        `pip install -r ../requirements.txt`
    *   To activate it (you'll need to run it every time you need the virtual env):
        `source env/bin/activate`
    *   To deactivate it:
        `deactivate`
        This works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate). Also, the virtual environment should be created only to run the python file. Docker images should first all be up and running.
*   If you encounter a "ModuleNotFoundError: No module named 'kafka

In [30]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [31]:
from elasticsearch import Elasticsearch

In [32]:
es_client = Elasticsearch('http://localhost:9200')

In [33]:
es_client.info()

ObjectApiResponse({'name': '4bde59e872a1', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4Ffnt1j6Q4OirjB8EpGFaQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [34]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [36]:
index_name = "course-questions"

es_client.indices.create(index=index_name, settings=index_settings['settings'], mappings=index_settings['mappings'])

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [41]:
from tqdm.auto import tqdm

In [50]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 427.50it/s]


In [51]:
query = q

In [62]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, size=search_query['size'], query=search_query['query'])
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs
    

In [65]:
elastic_search('how do I run kafka?')

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package

In [59]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minu

In [66]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [68]:
rag('how do i run kafka?')

'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n'

In [54]:
{}

{}

In [ ]:
\

In [ ]:
[]